In [ ]:
import sys
sys.path.append('../ASC-ODE/build/mechsystem')


from mass_spring import *

from pythreejs import *

import math

In [ ]:
mss = MassSpringSystem3d()
mss.gravity = (0,0,-9.81)


# demo
mA = mss.add (Mass(1, (1,0,0)))
mB = mss.add (Mass(2, (2,0,0)))
f1 = mss.add (Fix( (0,0,0)) )
mss.add (Spring(1, 200000, (f1, mA)))
mss.add (Spring(1, 100000, (mA, mB)))


# # chain
# mA = mss.add (Mass(50, (1,0,0)))
# mB = mss.add (Mass(30, (2,0,0)))
# mC = mss.add (Mass(10, (3,0,0)))
# mD = mss.add (Mass(2, (4,0,0)))
# mE = mss.add (Mass(2, (5,0,0)))
# f1 = mss.add (Fix( (0,0,0)) )
# mss.add (Spring(1, 2000, (f1, mA)))
# mss.add (Spring(1, 1000, (mA, mB)))
# mss.add (Spring(1, 1000, (mB, mC)))
# mss.add (Spring(1, 1000, (mC, mD)))
# mss.add (Spring(1, 1000, (mD, mE)))

# crane
# f1 = mss.add (Fix( (0,0,0)) )
# f2 = mss.add (Fix( (0,4,0)) )
# f3 = mss.add (Fix( (4,0,0)) )
# f4 = mss.add (Fix( (4,4,0)) )
# mA = mss.add (Mass(700, (1, 2, -1)))
# mB = mss.add (Mass(700, (3, 2, -1)))
# mss.add (Spring(2, 100000, (mA, mB)))
# mss.add (Spring(2, 10000, (f1, mA)))
# mss.add (Spring(2, 10000, (f2, mA)))
# mss.add (Spring(2, 10000, (f3, mB)))
# mss.add (Spring(2, 10000, (f4, mB)))

# spinning top
# f1 = mss.add (Fix( (0,0,0)) )
# mA = mss.add (Mass(100, (1,0,4)))
# mB = mss.add (Mass(100, (0,1,4)))
# mC = mss.add (Mass(100, (-1,0,4)))
# mD = mss.add (Mass(100, (0,0,-5)))

# R = 2.0
# H = 4.0
# Ax, Ay = R * math.cos(0), R * math.sin(0)
# mA = mss.add(Mass(100, (Ax, Ay, H)))
# Bx, By = R * math.cos(2*math.pi/3), R * math.sin(2*math.pi/3)
# mB = mss.add(Mass(100, (Bx, By, H)))
# Cx, Cy = R * math.cos(4*math.pi/3), R * math.sin(4*math.pi/3)
# mC = mss.add(Mass(100, (Cx, Cy, H)))

# mss.add (Spring(5, 5000000, (f1, mA)))
# mss.add (Spring(5, 5000000, (f1, mB)))
# mss.add (Spring(5, 5000000, (f1, mC)))
# mss.add (Spring(1, 100, (mA, mD)))
# mss.add (Spring(1, 100, (mB, mD)))
# mss.add (Spring(1, 100, (mC, mD)))
# mss.add (Spring(1, 1000000, (mA, mB)))
# mss.add (Spring(1, 1000000, (mB, mC)))
# mss.add (Spring(1, 1000000, (mC, mA)))

# mss.add(Spring(math.sqrt(R**2 + H**2), 500000, (f1, mA)))
# mss.add(Spring(math.sqrt(R**2 + H**2), 500000, (f1, mB)))
# mss.add(Spring(math.sqrt(R**2 + H**2), 500000, (f1, mC)))
# side_len = R * math.sqrt(3)
# mss.add(Spring(side_len, 1000000, (mA, mB)))
# mss.add(Spring(side_len, 1000000, (mB, mC)))
# mss.add(Spring(side_len, 1000000, (mC, mA)))


# mss.add (DistanceConstraint(1.0, (mA, mB)))
# mss.add (DistanceConstraint(1.0, (mB, mC)))
# mss.add (DistanceConstraint(1.0, (mC, mA)))
# mss.add (DistanceConstraint(1.0, (f1, mA)))
# mss.add (DistanceConstraint(1.0, (f1, mB)))
# mss.add (DistanceConstraint(1.0, (f1, mC)))
# mss.add (DistanceConstraint(1.0, (mA, mD)))
# mss.add (DistanceConstraint(1.0, (mB, mD)))
# mss.add (DistanceConstraint(1.0, (mC, mD)))


In [ ]:
masses = []
for m in mss.masses:
    masses.append(
        Mesh(SphereBufferGeometry(0.2, 16, 16),
             MeshStandardMaterial(color='red'),
             position=m.pos)) 

fixes = []
for f in mss.fixes:
    fixes.append(
        Mesh(SphereBufferGeometry(0.2, 32, 16),
             MeshStandardMaterial(color='blue'),
             position=f.pos)) 

springpos = []
for s in mss.springs:
    pA = mss[s.connectors[0]].pos
    pB = mss[s.connectors[1]].pos
    springpos.append ([ pA, pB ] ) 

springgeo = LineSegmentsGeometry(positions=springpos)
m2 = LineMaterial(linewidth=3, color='cyan')
springs = LineSegments2(springgeo, m2)    

axes = AxesHelper(1)

In [ ]:
view_width = 600
view_height = 400

camera = PerspectiveCamera( position=[10, 6, 10], aspect=view_width/view_height)
key_light = DirectionalLight(position=[0, 10, 10])
ambient_light = AmbientLight()

scene = Scene(children=[*masses, *fixes, springs, axes, camera, key_light, ambient_light])
controller = OrbitControls(controlling=camera)
renderer = Renderer(camera=camera, scene=scene, controls=[controller],
                    width=view_width, height=view_height)


renderer

In [ ]:
from time import sleep
for i in range(1000):
    mss.simulate (0.02, 100)
    for m,mvis in zip(mss.masses, masses):
        mvis.position = (m.pos[0], m.pos[1], m.pos[2])

    springpos = []
    for s in mss.springs:
        pA = mss[s.connectors[0]].pos
        pB = mss[s.connectors[1]].pos
        springpos.append ([ pA, pB ]) 
    springs.geometry = LineSegmentsGeometry(positions=springpos)
    sleep(0.01)